In [25]:
import pandas as pd

data = pd.read_excel("preds.xlsx")
data

,user_id,True_val,Preds
0,1,5.0,5
1,1,5.0,4
2,1,NaN,4
3,1,4.5,4
4,1,2.0,4
5,1,NaN,4
6,1,2.0,5
7,1,2.0,5
8,1,NaN,5
9,1,2.0,5


In [70]:
precisions = []
recalls = []
aps = []
k = 10

# Calculate precision@k and recall@k for each user
users = data["user_id"].unique().tolist()
for user_id in users:
    # Get only relevant rows for the current user, 
    user_data = data[data["user_id"]==user_id]
    
    # Get number of relevant items (relevant= items with 4 or higher rank)    
    number_of_relevant_items = user_data.dropna()["True_val"].apply(lambda x: 1 if x>=4 else 0).sum() 
    
    # Deal with users without any items in the test set (if there are any)
    if number_of_relevant_items==0:
        continue
    
    #sort the predictions and select the top k 
    user_data = user_data.sort_values("Preds", ascending=False).head(k)
    
    # Drop rows with no True value
    user_data = user_data.dropna()
    
    # Trim values (above-equal to 4 means 1, else 0)
    user_data["Preds"] = user_data["Preds"].apply(lambda x: 1 if x>=4 else 0)
    user_data["True_val"] = user_data["True_val"].apply(lambda x: 1 if x>=4 else 0)
    
    # Calculate hits
    user_data["hits"] = user_data.apply(lambda row: 1 if row["Preds"] == row["True_val"] else 0, axis=1)
    
    # Calculate precision and recall
    recall = user_data["hits"].sum() / number_of_relevant_items
    precision = user_data["hits"].sum() / k
    
    precisions.append(precision)
    recalls.append(recall)
    
    # Calculate AP@k
    precisions_at_i = []
    for i in range(1,k+1):
        ap_precision = user_data.head(i)["hits"].sum() / i
        precisions_at_i.append(ap_precision)
    # average accross i's up to k
    ap_at_k = sum(precisions_at_i) / len(precisions_at_i)
    aps.append(ap_at_k) 
    
    
# Average precision@k and recall@k over all the users
recall_at_k = sum(recalls) / len(recalls)
precision_as_k = sum(precisions) / len(precisions)
map_at_k = sum(aps) / len(aps)
print(f"Precision@{k} is: {precision_as_k}")
print(f"Recall@{k} is {recall_at_k}")
print(f"MAP@{k} is {map_at_k}")

Precision@10 is: 0.2
Recall@10 is 0.5
MAP@10 is 0.40246031746031746
